In [1]:
import os
import time
import sys
import itertools
import json
import librosa as lr
import librosa.display as lrd
import numpy as np
import os.path as osp

import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.metrics import mean_squared_error

%matplotlib inline

sys.path.append("..")
import pickle as pk


In [2]:
res_path = '/home/christie/results/model_190524_0857_conv_tcn_bf6186/results_190524_1322.pkl'

### Phase 2 - ds1
# tcn-mag: /home/christie/results/model_190517_1742_conv_tcn_182082/results_190523_0016.pkl
# tcn-db: /home/christie/results/model_190517_1547_conv_tcn_a30f3a/results_190523_0026.pkl

# gru-mag: /home/rmicci18/results/model_190516_1506_conv_gru_65c809/results_190523_1159.pkl
# gru-db: /home/rmicci18/results/model_190516_1111_conv_gru_e39df2/results_190521_1950.pkl

# rnn-mag: /home/rmicci18/results/model_190516_1654_conv_rnn_fb13dc/results_190523_1232.pkl
# rnn-db: /home/christie/results/model_190516_2012_conv_rnn_ce7efe/results_190524_1055.pkl


### Phase 2 - ds2 
# ae-db: /home/christie/results/model_190521_1547_conv_ae_a4bc31/results_190523_0853.pkl

# tcn-mag: /home/christie/results/model_190521_1016_conv_tcn_182082/results_190523_1153.pkl
# tcn-db: /home/rmicci18/results/model_190521_1011_conv_tcn_a30f3a/results_190522_1853.pkl

# rnn-mag: /home/rmicci18/results/model_190519_1941_conv_rnn_fb13dc/results_190522_2330.pkl
# rnn-db: /home/christie/results/model_190519_1927_conv_rnn_fb13dc/results_190524_1125.pkl




### Tests tcn-db
# (conv_tcnn2d8_2layers_db.h5): /results/model_190523_1734_conv_tcn_da0f6c/results_190524_0532.pkl
# (conv_tcnn4d2_1layer_db.h5): /results/model_190523_1926_conv_tcn_1e4ec0/results_190524_0539.pkl
# (conv_tcnn4d2_1layer_mag.h5): /results/model_190524_0857_conv_tcn_bf6186/results_190524_1322.pkl

In [3]:
with open(res_path,'rb') as f: 
    res = pk.load(f)
res


mse  \
filepath                                           noise_variation                                                
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.066509   
                                                   (<libs.processing.take_file_as_noise object at ...  0.039038   
                                                   (<libs.processing.take_file_as_noise object at ...  0.085229   
                                                   (<libs.processing.take_file_as_noise object at ...  0.041371   
                                                   (<libs.processing.take_file_as_noise object at ...  0.045383   
                                                   (<libs.processing.take_file_as_noise object at ...  0.035720   
                                                   (<libs.processing.take_file_as_noise object at ...  0.064808   
                                                   (<libs.processing.take_file_as_noise object at ...  0.036539   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.073095   
                                                   (<libs.processing.take_file_as_noise object at ...  0.047502   
                                                   (<libs.processing.take_file_as_noise object at ...  0.113773   
                                                   (<libs.processing.take_file_as_noise object at ...  0.051390   
                                                   (<libs.processing.take_file_as_noise object at ...  0.054976   
                                                   (<libs.processing.take_file_as_noise object at ...  0.044982   
                                                   (<libs.processing.take_file_as_noise object at ...  0.082604   
                                                   (<libs.processing.take_file_as_noise object at ...  0.045571   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.044922   
                                                   (<libs.processing.take_file_as_noise object at ...  0.029691   
                                                   (<libs.processing.take_file_as_noise object at ...  0.083158   
                                                   (<libs.processing.take_file_as_noise object at ...  0.030865   
                                                   (<libs.processing.take_file_as_noise object at ...  0.037546   
                                                   (<libs.processing.take_file_as_noise object at ...  0.028494   
                                                   (<libs.processing.take_file_as_noise object at ...  0.050939   
                                                   (<libs.processing.take_file_as_noise object at ...  0.028984   
/data/riccardo_datasets/npr_news/ds2/test/NPR_N... (<libs.processing.take_file_as_noise object at ...  0.048729   
                                                   (<libs.processing.take_file_as_noise object at ...  0.029450   
                                                   (<libs.processing.take_file_as_noise object at ...  0.081653   
                                                   (<libs.processing.take_file_as_noise object at ...  0.030128   
                                                   (<libs.processing.take_file_as_noise object at ...  0.039966   
                                                   (<libs.processing.take_file_as_noise object at ...  0.027922   
                                                   (<libs.processing.take_file_as_noise object at ...  0.055607   
                                                   (<libs.processing.take_file_as_noise object at ...  0.028635   

                                                                                                             sdr  \
filepath                                           noise_variation               

In [4]:
res.shape
nb_occur = res['mse'].shape[0]
print(nb_occur)

32


In [5]:
ana_types = {'mse', 'sdr', 'sir', 'sar', 'stoi', 'pesq'}
snr = {5, 15}
stat_noises = {'narrow_seen','narrow_unseen', 'broad_seen', 'broad_unseen'}


In [6]:
# metrics averaging on noises, for each snr

for a in ana_types:
    print('metrics :',a)
    analysis_5 =  [res[a][2*im] for im in range(nb_occur//2) ]
    analysis_15 = [res[a][2*im+1] for im in range(nb_occur//2) ]
    # print("mse_5 :", mse_5, ",\n mse_15 :", mse_15)

    mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
    mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
    print("mean_5: {}, std_5: {}".format(mean_5, std_5))
    print("mean_15 {}, std_15: {}".format(mean_15, std_15))

metrics : mse
mean_5: 0.06430601458370351, std_5: 0.020286441188854643
mean_15 0.036017541221050296, std_15: 0.007699373821195693
metrics : sar
mean_5: 15.976735978940626, std_5: 2.7223313661147372
mean_15 20.3909326932895, std_15: 0.7756560833597586
metrics : pesq
mean_5: 2.6508884131908417, std_5: 0.1052104091558581
mean_15 2.900293419758479, std_15: 0.05316859123319247
metrics : sdr
mean_5: 13.845235528913157, std_5: 2.359532451763128
mean_15 19.793515364112466, std_15: 0.9502868543606323
metrics : sir
mean_5: 18.927401185882395, std_5: 3.0868464009503414
mean_15 29.0819339416379, std_15: 2.48255706999081
metrics : stoi
mean_5: 0.8645977711679735, std_5: 0.0309138543889031
mean_15 0.9025067179172935, std_15: 0.029817638000133814


In [8]:
# metrics for each noise, for each snr

for a in ana_types:
    print('\n Metrics :', a)
    for n, noise in enumerate(stat_noises):
        print('noise :', noise)
        analysis_5 =  [res[a][8*im+2*n] for im in range(nb_occur//8) ]
        analysis_15 = [res[a][8*im+2*n+1] for im in range(nb_occur//8) ]
    #     print(analysis_5)

        mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
        mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
        print("mean_5: {}, std_5: {}".format(mean_5, std_5))
        print("mean_15 {}, std_15: {}".format(mean_15, std_15))
    


 Metrics : sdr
noise : narrow_seen
mean_5: 16.765855189860872, std_5: 0.4016806165303759
mean_15 25.63091154466464, std_15: 0.17206469515271827
noise : narrow_unseen
mean_5: 16.27593767267319, std_5: 0.22441415431080358
mean_15 25.51107035992424, std_15: 0.07929988257221865
noise : broad_unseen
mean_5: 16.45260990676349, std_5: 0.07660145230398285
mean_15 24.427996669753473, std_15: 0.549661116341607
noise : broad_seen
mean_5: 16.23704942308197, std_5: 0.34795587760461366
mean_15 26.032445540182273, std_15: 0.221932877844905

 Metrics : stoi
noise : narrow_seen
mean_5: 0.8552286221206982, std_5: 0.044419957958412026
mean_15 0.9039885770217821, std_15: 0.04211234768405253
noise : narrow_unseen
mean_5: 0.8869920452993215, std_5: 0.021314725403272727
mean_15 0.9389526021692716, std_15: 0.012427469033468197
noise : broad_unseen
mean_5: 0.8674130730382239, std_5: 0.01137769424478613
mean_15 0.9052892815870985, std_15: 0.009966357823886551
noise : broad_seen
mean_5: 0.8883173181577042, std_

In [9]:
# metrics for each noise, for each snr
for n, noise in enumerate(stat_noises):
    print('\n Noise no :', noise)
    for a in ana_types:
        print('metrics :', a)
    
        analysis_5 =  [res[a][8*im+2*n] for im in range(nb_occur//8) ]
        analysis_15 = [res[a][8*im+2*n+1] for im in range(nb_occur//8) ]
    #     print(analysis_5)

        mean_5, std_5 = np.mean(analysis_5), np.std(analysis_5)
        mean_15, std_15 = np.mean(analysis_15), np.std(analysis_15)
        print("mean_5: {}, std_5: {}".format(mean_5, std_5))
        print("mean_15 {}, std_15: {}".format(mean_15, std_15))


 Noise no : narrow_seen
metrics : sdr
mean_5: 16.765855189860872, std_5: 0.4016806165303759
mean_15 25.63091154466464, std_15: 0.17206469515271827
metrics : stoi
mean_5: 0.8552286221206982, std_5: 0.044419957958412026
mean_15 0.9039885770217821, std_15: 0.04211234768405253
metrics : sir
mean_5: 19.71335935744274, std_5: 0.7564364434967956
mean_15 33.125795327095126, std_15: 0.30390239400460217
metrics : sar
mean_5: 19.917215398656474, std_5: 0.02586024322524061
mean_15 26.49140175515183, std_15: 0.27566821778928485
metrics : pesq
mean_5: 2.950516621271769, std_5: 0.03915095329284668
mean_15 3.695655981699626, std_15: 0.006555159886677986
metrics : mse
mean_5: 0.02310197437493053, std_5: 0.0011455354035841397
mean_15 0.007217007339521337, std_15: 0.0001635860948813149

 Noise no : narrow_unseen
metrics : sdr
mean_5: 16.27593767267319, std_5: 0.22441415431080358
mean_15 25.51107035992424, std_15: 0.07929988257221865
metrics : stoi
mean_5: 0.8869920452993215, std_5: 0.021314725403272727


In [10]:
###########################################

In [ ]:
# mse

mse_5 =  [res['mse'][2*im] for im in range(nb_occur//2) ]#np.zeros()
mse_15 = [res['mse'][2*im+1] for im in range(nb_occur//2) ]
# print("mse_5 :", mse_5, ",\n mse_15 :", mse_15)

mean_mse_5, std_mse_5 = np.mean(mse_5), np.std(mse_5)
mean_mse_15, std_mse_15 = np.mean(mse_15), np.std(mse_15)
print("mean_mse_5: {}, std_mse_5: {}".format(mean_mse_5, std_mse_5))
print("mean_mse_15 {}, std_mse_15: {}".format(mean_mse_15, std_mse_15))

In [ ]:
# sdr

sdr_5 =  [res['sdr'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sdr_15 = [res['sdr'][2*im+1] for im in range(nb_occur//2) ]
# print("sdr_5 :", sdr_5, ",\n sdr_15 :", sdr_15)

mean_sdr_5, std_sdr_5 = np.mean(sdr_5), np.std(sdr_5)
mean_sdr_15, std_sdr_15 = np.mean(sdr_15), np.std(sdr_15)
print("mean_sdr_5: {}, std_sdr_5: {}".format(mean_sdr_5, std_sdr_5))
print("mean_sdr_15 {}, std_sdr_15: {}".format(mean_sdr_15, std_sdr_15))

In [ ]:
# sir

sir_5 =  [res['sir'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sir_15 = [res['sir'][2*im+1] for im in range(nb_occur//2) ]
# print("sir_5 :", sir_5, ",\n sir_15 :", sir_15)

mean_sir_5, std_sir_5 = np.mean(sir_5), np.std(sir_5)
mean_sir_15, std_sir_15 = np.mean(sir_15), np.std(sir_15)
print("mean_sir_5: {}, std_sir_5: {}".format(mean_sir_5, std_sir_5))
print("mean_sir_15 {}, std_sir_15: {}".format(mean_sir_15, std_sir_15))

In [ ]:
# sar

sar_5 =  [res['sar'][2*im] for im in range(nb_occur//2) ]#np.zeros()
sar_15 = [res['sar'][2*im+1] for im in range(nb_occur//2) ]
# print("sar_5 :", sar_5, ",\n sar_15 :", sar_15)

mean_sar_5, std_sar_5 = np.mean(sar_5), np.std(sar_5)
mean_sar_15, std_sar_15 = np.mean(sar_15), np.std(sar_15)
print("mean_sar_5: {}, std_sar_5: {}".format(mean_sar_5, std_sar_5))
print("mean_sar_15 {}, std_sar_15: {}".format(mean_sar_15, std_sar_15))

In [ ]:
# stoi

stoi_5 =  [res['stoi'][2*im] for im in range(nb_occur//2) ]#np.zeros()
stoi_15 = [res['stoi'][2*im+1] for im in range(nb_occur//2) ]
# print("stoi_5 :", stoi_5, ",\n stoi_15 :", stoi_15)

mean_stoi_5, std_stoi_5 = np.mean(stoi_5), np.std(stoi_5)
mean_stoi_15, std_stoi_15 = np.mean(stoi_15), np.std(stoi_15)
print("mean_stoi_5: {}, std_stoi_5: {}".format(mean_stoi_5, std_stoi_5))
print("mean_stoi_15 {}, std_stoi_15: {}".format(mean_stoi_15, std_stoi_15))

In [ ]:
# pesq

pesq_5 =  [res['pesq'][2*im] for im in range(nb_occur//2) ]#np.zeros()
pesq_15 = [res['pesq'][2*im+1] for im in range(nb_occur//2) ]
# print("pesq_5 :", pesq_5, ",\n pesq_15 :", pesq_15)

mean_pesq_5, std_pesq_5 = np.mean(pesq_5), np.std(pesq_5)
mean_pesq_15, std_pesq_15 = np.mean(pesq_15), np.std(pesq_15)
print("mean_pesq_5: {}, std_pesq_5: {}".format(mean_pesq_5, std_pesq_5))
print("mean_pesq_15 {}, std_pesq_15: {}".format(mean_pesq_15, std_pesq_15))
